# **ME LO DIO CHAT PERO TADRA AÑOS EN CORRER**

quiero hacer esto con NMF, MDS, ISOMAP, T_SNE y UMAP para hacer una comparacion. Actualmente lo tengo asi para PCA 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, NMF
from sklearn.manifold import MDS, Isomap, TSNE
import umap
import numpy as np
import pickle
import os

def unpickle(file_path):
    """Carga un archivo pickle de Python y devuelve el diccionario."""
    with open(file_path, 'rb') as fo:
        return pickle.load(fo, encoding='bytes')

# Ruta a la carpeta donde tienes la versión Python de CIFAR-100
cifar100_path = r"/home/heri/Documents/GitHub/Data-Mining/07-Reduccion-de-Dimensionalidad/02-Ejercicio/cifar-10-batches-py"

# Carga de los datos de entrenamiento y metadatos
train_dict = unpickle(os.path.join(cifar100_path, 'data_batch_1'))
meta_dict  = unpickle(os.path.join(cifar100_path, 'batches.meta'))

class1 = 'bird'
class2 = 'automobile'
class3 = 'cat'
class4 = 'deer'

# Decodificar nombres de clases
fine_label_names = [name.decode('utf-8') for name in meta_dict[b'label_names']]

# Obtener los índices de las clases class1 y class2
idx_class1 = fine_label_names.index(class1)
idx_class2 = fine_label_names.index(class2)
idx_class3 = fine_label_names.index(class3)
idx_class4 = fine_label_names.index(class4)

# Convertir datos y etiquetas a numpy
data = train_dict[b'data']               # (50000, 3072)
labels = np.array(train_dict[b'labels'])

# Filtrar solo las muestras de class1, class2, class3 y class4
mask = np.isin(labels, [idx_class1, idx_class2, idx_class3, idx_class4])
X = data[mask]
y = labels[mask]

# --- Métodos de reducción de dimensionalidad a comparar ---
reducers = {
    'PCA': PCA(n_components=2),
    'NMF': NMF(n_components=2, init='random', random_state=42, max_iter=300),
    'MDS': MDS(n_components=2, random_state=42, normalized_stress='auto'),
    'ISOMAP': Isomap(n_components=2, n_neighbors=5),  # Ajusta n_neighbors según tus datos
    'T-SNE': TSNE(n_components=2, random_state=42, n_iter=300, learning_rate='auto'),
    'UMAP': umap.UMAP(n_components=2, random_state=42)
}

# --- Pipeline de escalado ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# --- Generar las proyecciones y visualizaciones ---
plt.figure(figsize=(15, 12))
plot_index = 1

for name, reducer in reducers.items():
    # Usar X (no escalado) para NMF, X_scaled para los demás
    if name == 'NMF':
        X_reduced = reducer.fit_transform(X)
    else:
        X_reduced = reducer.fit_transform(X_scaled)

    # Crear el subplot
    plt.subplot(2, 3, plot_index)

    # Graficar cada clase
    masks = {
        class1: (y == idx_class1),
        class2: (y == idx_class2),
        class3: (y == idx_class3),
        class4: (y == idx_class4)
    }
    alphas = [0.3] * len(masks)  # Asegura que todas las clases tengan la misma transparencia

    for i, (label, mask) in enumerate(masks.items()):
        plt.scatter(X_reduced[mask, 0], X_reduced[mask, 1], marker='o', label=label, alpha=alphas[i])

    plt.xlabel(f'{name} Dimension 1')
    plt.ylabel(f'{name} Dimension 2')
    plt.title(f'Proyección {name}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    plot_index += 1

plt.show()